# Allen-Cahn 方程求解

## 环境安装与依赖

本案例使用 **MindSpore >= 2.0.0** 运行，支持函数式编程接口（如 `mindspore.jit`、`mindspore.jit_class`、`mindspore.data_sink`），以实现高效的神经网络训练。安装方法请参考 [MindSpore 官方安装指南](https://www.mindspore.cn/install)。

此外，需要安装 **MindFlow >= 0.1.0**，这是 MindSpore 的流体模拟套件，提供 PDE 求解和 PINNs 方法的支持。如果尚未安装，请根据硬件环境（GPU 或 NPU）选择合适的版本并执行以下代码安装。

本案例在 GPU 环境下运行，若使用 NPU，请注释掉 GPU 相关安装代码。

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## 概述

Allen-Cahn 方程是一种非线性偏微分方程（PDE），以 John W. Cahn 和 Sam Allen 命名，广泛应用于相场模型，描述多组分合金系统中的相分离过程，如有序-无序转变。其形式为一个反应-扩散方程，刻画标量状态变量

$$ u(x, t) $$

在时空中的演化。本案例利用 **MindFlow** 流体模拟套件，基于物理驱动神经网络（PINNs）方法求解 Allen-Cahn 方程。

传统数值方法（如有限元法 FEM 和有限差分法 FDM）在求解 PDE 时面临建模复杂、网格划分繁琐、计算成本高等问题。PINNs 结合神经网络的通用逼近能力和自动微分技术，通过最小化 PDE 残差及边界/初始条件损失，直接学习解的映射

$$ (x, t) \mapsto u(x, t) $$

无需显式网格划分，显著提高效率。

本案例展示了如何使用 PINNs 求解 Allen-Cahn 方程，包括数据集生成、模型构建、损失函数定义、训练优化及结果可视化，为非线性 PDE 求解提供了一种高效的替代方案。

## 问题描述

Allen-Cahn 方程的数学形式为：

$$
\frac{\partial u}{\partial t} = d \frac{\partial^2 u}{\partial x^2} + 5(u - u^3), \quad x \in [-1, 1], \quad t \in [0, 1], \quad d = 0.001
$$

其中：
- $ u(x, t) $ 是状态变量，表示相场量。
- $ d = 0.001 $ 是扩散系数。
- $ f(u) = 5(u - u^3) $ 是非线性反应项，驱动相分离行为。

边界条件（Dirichlet）：

$$
u(-1, t) = -1, \quad u(1, t) = -1, \quad t \in [0, 1]
$$

初始条件：

$$
u(x, 0) = x^2 \cos(\pi x), \quad x \in [-1, 1]
$$

本案例的目标是通过 PINNs 方法，训练神经网络学习映射

$$(x, t) \mapsto u(x, t)$$

使之满足 Allen-Cahn 方程及其边界/初始条件。PINNs 通过构造损失函数（基于 PDE 残差），利用自动微分计算导数，无需传统网格划分，直接优化神经网络参数。

## 技术路径

MindFlow 求解 Allen-Cahn 方程的流程基于 PINNs 方法，具体步骤如下：

1. **创建数据集**：根据定义域随机采样训练数据，并加载测试数据集用于评估。
2. **构建模型**：使用多尺度全连接神经网络，支持硬约束边界/初始条件。
3. **优化器**：采用 Adam 优化器，设置学习率和衰减策略，优化网络参数。
4. **定义 Allen-Cahn 问题**：通过 `AllenCahn` 类定义 PDE、边界/初始条件和损失函数，利用 SymPy 和 MindSpore 实现自动微分和损失计算。
5. **模型训练**：基于 PDE 残差损失进行训练，定期评估 L2 误差，保存最优模型和检查点。
6. **模型推理与可视化**：使用训练好的模型对定义域进行推理，生成 $ u(x, t) $ 的时空分布，并可视化结果。

In [ ]:
import os
import math
import time

import numpy as np

import mindspore
from mindspore import context, nn, ops, Tensor, jit, set_seed
from mindspore import dtype as mstype
from mindspore import load_checkpoint, load_param_into_net, save_checkpoint
from mindflow.pde import PDEWithLoss, sympy_to_mindspore
from mindflow.utils import load_yaml_config
from mindflow.loss import get_loss_metric

from sympy import diff, symbols, Function

## 依赖包与源代码

上述代码导入了求解 Allen-Cahn 方程所需的 MindSpore 和 MindFlow 核心模块，以及 SymPy 用于符号计算。[src](./src/) 包包含自定义的模型和工具函数，包括：

- `MultiScaleFCSequentialOutputTransform`：多尺度全连接神经网络，支持输出变换。
- `create_training_dataset` 和 `create_test_dataset`：生成训练和测试数据集。
- `visual` 和 `calculate_l2_error`：可视化结果和计算 L2 误差。

这些模块共同支持 PINNs 方法的实现。

In [ ]:
from src import MultiScaleFCSequentialOutputTransform
from src import create_training_dataset, create_test_dataset, visual, calculate_l2_error

# 设置随机种子以确保结果可重现
set_seed(123456)
np.random.seed(123456)

In [ ]:
# 设置运行上下文，使用 GPU 训练
context.set_context(mode=context.GRAPH_MODE, device_target="GPU", device_id=0)

从configs文件夹下导入配置文件

注：配置文件默认为训练第一个数据集的配置，配置文件当中默认训练为 5000 轮，训练第二个数据集请将 test_dataset_path 改为 "../dataset/2"

In [ ]:
# 加载配置文件
config = load_yaml_config('./configs/allen_cahn_cfg.yaml')

## 创建数据集

训练数据集通过随机采样生成，包括：
- **定义域内部点**：8192 个点，用于计算 PDE 残差损失。
- **边界点**：800 个点 $ x = \pm 1 $ 用于边界条件。
- **初始点**：400 个点 $ t = 0 $ 用于初始条件。

采样方法为均匀随机采样（`uniform`），确保点分布覆盖整个时空定义域。测试数据集从指定路径加载，用于评估模型精度（L2 误差）。

数据集目录：[allen_cahn/dataset](./allen_cahn/dataset)。

In [ ]:
# 创建训练数据集
ac_train_dataset = create_training_dataset(config)
train_dataset = ac_train_dataset.create_dataset(batch_size=config["train_batch_size"],
                                                shuffle=True,
                                                prebatched_data=True,
                                                drop_remainder=True)
# 创建测试数据集
inputs, label = create_test_dataset(config["test_dataset_path"])

## 构建模型

本案例使用 `MultiScaleFCSequentialOutputTransform` 或配置文件中的 `MLP_with_Residual` 模型，具体配置如下：
- **结构**：6 层全连接网络，每层 128 个神经元。
- **激活函数**：`tanh`，适合平滑非线性变换。
- **残差连接**：启用，增强深层网络的训练稳定性。
- **输入/输出**：输入为 $ (x, t) $ ，输出为 $ u(x, t) $。

若启用检查点加载（`load_ckpt=True`），模型将从指定路径恢复预训练参数，继续训练或推理。

In [ ]:
# 定义模型
model = MultiScaleFCSequentialOutputTransform(in_channels=config["model"]["in_channels"],
                                              out_channels=config["model"]["out_channels"],
                                              layers=config["model"]["layers"],
                                              neurons=config["model"]["neurons"],
                                              residual=config["model"]["residual"],
                                              act=config["model"]["activation"],
                                              num_scales=1)
# 加载 checkpoint
if config["load_ckpt"]:
    param_dict = load_checkpoint(config["load_ckpt_path"])
    load_param_into_net(model, param_dict)

## 优化器

优化器采用 Adam 算法，初始学习率从配置文件中读取（默认 0.0001）。支持学习率预热（`warmup_epochs`）和衰减（`gamma`），以提高训练稳定性和收敛速度。Adam 优化器管理模型的所有可训练参数，通过最小化损失函数更新网络权重。

In [ ]:
# 定义优化器
optimizer = nn.Adam(model.trainable_params(),
                    config["optimizer"]["initial_lr"])

## 定义模型

`AllenCahn` 类继承自 `PDEWithLoss`，用于定义 Allen-Cahn 方程及其损失函数。主要功能包括：
- **PDE 定义**：通过 SymPy 定义方程 $u_t - 0.001 u_{xx} - 5(u - u^3) = 0$ 并转换为 MindSpore 计算节点。
- **输出变换**：通过 $u(x, t) = x^2 \cos(\pi x) + t (1 - x^2) \cdot \text{out}$ 硬编码边界条件 $u(\pm 1, t) = -1$ 和初始条件 $u(x, 0) = x^2 \cos(\pi x)$
- **损失计算**：仅计算 PDE 残差损失（均方误差），无需显式边界/初始条件损失。

这种硬约束策略简化了损失函数设计，提高训练效率。

In [ ]:
class AllenCahn(PDEWithLoss):
    """
    Allen-Cahn方程的PDE损失类，用于定义方程并计算物理驱动的神经网络损失。
    继承自PDEWithLoss，通过SymPy定义PDE并转换为MindSpore计算节点，支持输出变换以满足边界条件。
    """

    def __init__(self, model, loss_fn="mse"):
        """
        初始化Allen-Cahn方程的PDE模型。
        参数：
            model: 神经网络模型。
            loss_fn: 损失函数，默认为均方误差（mse），可传入字符串或自定义函数。
        """
        # 定义符号变量 x（空间）和 t（时间），以及函数 u(x, t)
        self.x, self.t = symbols("x t")
        self.u = Function("u")(self.x, self.t)
        self.in_vars = [self.x, self.t]
        self.out_vars = [self.u]
        # 根据输入设置损失函数，字符串则使用预定义损失函数
        if isinstance(loss_fn, str):
            self.loss_fn = get_loss_metric(loss_fn)
        else:
            self.loss_fn = loss_fn
        # 将PDE方程转换为MindSpore计算节点
        self.pde_nodes = sympy_to_mindspore(
            self.pde(), self.in_vars, self.out_vars)
        # 设置模型的输出变换函数以满足边界条件
        model.set_output_transform(self.output_transform)
        super(AllenCahn, self).__init__(model, self.in_vars, self.out_vars)

    def output_transform(self, x, out):
        """
        输出变换函数，调整神经网络输出以满足Allen-Cahn方程的边界条件。
        参数：
            x: 输入张量，包含空间坐标 x 和时间 t。
            out: 神经网络的原始输出。
        返回：
            调整后的输出，结合边界条件（如 x=±1 时值为零）。
        """
        return x[:, 0:1] ** 2 * ops.cos(np.pi * x[:, 0:1]) + x[:, 1:2] * (1 - x[:, 0:1] ** 2) * out

    def force_function(self, u):
        """
        定义Allen-Cahn方程的非线性力项。
        参数：
            u: 函数值 u(x, t)。
        返回：
            非线性项 5 * (u - u^3)。
        """
        return 5 * (u - u ** 3)

    def pde(self):
        """
        定义Allen-Cahn偏微分方程。
        返回：
            包含PDE损失的字典，形式为 u_t - d * u_xx - f(u)，其中 d=0.001。
        """
        d = 0.001
        loss_1 = (
            self.u.diff(self.t)  # u 对时间 t 的一阶导数
            - d * diff(self.u, (self.x, 2))  # 负 d 乘以 u 对 x 的二阶导数
            - self.force_function(self.u)  # 减去非线性力项
        )
        return {"loss_1": loss_1}

    def get_loss(self, pde_data):
        """
        计算PDE残差损失。
        参数：
            pde_data: 输入数据，包含空间和时间坐标。
        返回：
            PDE损失值，基于PDE残差与零的均方误差。
        """
        pde_res = ops.Concat(1)(self.parse_node(
            self.pde_nodes, inputs=pde_data))  # 拼接PDE节点计算结果
        pde_loss = self.loss_fn(
            pde_res, Tensor(np.array([0.0]).astype(
                np.float32), mstype.float32)  # 计算与零的损失
        )

        return pde_loss

## 模型训练

训练过程利用 **MindSpore >= 2.0.0** 的函数式编程范式，通过以下步骤实现：
- **前向传播**：通过 `forward_fn` 计算 PDE 残差损失。
- **初始化问题**：创建 `AllenCahn` 实例，定义 PDE 和损失函数。
- **梯度计算**：使用 `ops.value_and_grad` 获取损失及其梯度。
- **加速训练**：采用 JIT 编译（`@jit`）和 `data_sink` 优化数据流，提升训练效率。
- **训练循环**：执行 epochs 轮训练（由配置文件指定），每批次 400 个采样点，定期评估 L2 误差，保存最优模型和检查点。

训练仅优化 PDE 残差损失，边界和初始条件通过 `output_transform` 硬编码，无需显式损失项。训练过程记录每个 epoch 的损失和耗时，保存最优模型（`ac-optimal.ckpt`）和定期检查点（`ac-<epoch>.ckpt`）。

In [ ]:
def train():
    '''训练和评估神经网络，用于求解 Allen-Cahn 方程'''
    # 初始化 Allen-Cahn 问题，传入神经网络模型
    problem = AllenCahn(model)
    
    # 初始化损失缩放器
    loss_scaler = None

    # 计算 PDE 残差损失
    def forward_fn(pde_data):
        # 调用 AllenCahn 类的 get_loss 方法，计算 PDE 残差损失
        loss = problem.get_loss(pde_data)
        return loss

    # 定义梯度计算函数，基于 forward_fn 获取损失值及其梯度
    grad_fn = ops.value_and_grad(
        forward_fn,            # 前向传播函数
        None,                 # 不指定特定参数（默认对所有参数求梯度）
        optimizer.parameters, # 优化器管理的神经网络参数
        has_aux=False         # 不返回辅助输出，仅返回损失和梯度
    )

    # 使用 JIT（即时编译）加速训练过程
    @jit
    def train_step(pde_data):
        loss, grads = grad_fn(pde_data)
        loss = ops.depend(loss, optimizer(grads))
        return loss
    
    # 跟踪最优模型
    min_loss = math.inf

    epochs = config["train_epochs"]
    steps_per_epochs = train_dataset.get_dataset_size()
    # 使用 data_sink 加速数据处理，将 train_step 绑定到训练数据集
    sink_process = mindspore.data_sink(train_step, train_dataset, sink_size=1)
    
    # 开始训练循环
    for epoch in range(1, 1 + epochs):
        time_beg = time.time()
        model.set_train(True)
        for _ in range(steps_per_epochs):
            step_train_loss = sink_process()
        print(
            f"epoch: {epoch} train loss: {step_train_loss} epoch time: {(time.time() - time_beg)*1000 :.3f} ms")
        
        # 设置模型为评估模式（禁用 dropout 等）
        model.set_train(False)  
        if epoch % config["eval_interval_epochs"] == 0:
            # 计算模型预测与真实解的 L2 误差
            calculate_l2_error(model, inputs, label, config["train_batch_size"])
        
        # 保存最优模型（基于最小损失）
        if config["save_ckpt"] and step_train_loss < min_loss:
            min_loss = step_train_loss
            ckpt_name = "ac-optimal.ckpt"
            # 保存最优模型检查点
            save_checkpoint(model, os.path.join(config["save_ckpt_path"], ckpt_name))
        
        # 按配置的保存间隔保存检查点
        if epoch % config["save_checkpoint_epochs"] == 0 and config["save_ckpt"]:
            if not os.path.exists(os.path.abspath(config["save_ckpt_path"])):
                config["save_ckpt_path"] = os.path.abspath("./ckpt")
            ckpt_name = "ac-{}.ckpt".format(epoch)
            save_checkpoint(model, os.path.join(config["save_ckpt_path"], ckpt_name))
            print(os.path.join(config["save_ckpt_path"], ckpt_name))
    print(f'final min_loss: {min_loss}')

In [ ]:
start_time = time.time()
train()
print("End-to-End total time: {} s".format(time.time() - start_time))

## 模型推理与可视化

训练完成后，加载最优模型进行推理，预测整个定义域的解。
可视化结果包括：
- **时空分布图**：展示 $ u(x, t) $ 在整个定义域的分布，颜色表示 $ u $ 值。
- **时间截面图**：在特定时间点绘制 $ u(x) $ 曲线，反映相场演化。

In [ ]:
# visualization
if config["save_ckpt"]:
    optimal_param = load_checkpoint(os.path.join(config["save_ckpt_path"], "ac-optimal.ckpt"))
    load_param_into_net(model, optimal_param)
    
epochs = config["train_epochs"]
save_path = f'images/gpu_result_dataset_{config["test_dataset_path"].split("/")[-1]}.jpg'
visual(model, epochs=epochs, resolution=config["visual_resolution"],path=save_path)

### 数据集一可视化
![dataset_1](./images/gpu_result_dataset_1.jpg)

### 数据集二可视化

![dataset_2](./images/gpu_result_dataset_2.jpg)

### 分析

* **时空分布图** $u(x, t)$ 在 $x = \pm 1$ 始终为 $-1$，中心区域随 $t$ 增加形成 $u \approx 1$ 并逐渐变窄，符合 Allen-Cahn 方程的**相分离特性**(系统自发分裂为 $u \approx \pm 1$ 的稳定相)。

* **时间截面图** $u(x)$ 从宽峰逐步变窄，中心 $u \approx 1$，两侧 $u \approx -1$，反映出动态演化过程，即非线性项驱动界面向稳定态收敛。

* 图像特征与 Allen-Cahn 方程的理论行为高度一致。